In [27]:
#install tesseract
#!sudo apt install tesseract-ocr

In [28]:
#install pytesseract (python wrapper)
#!pip install pytesseract

In [29]:
#mount drive to get dashcam video
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import pytesseract
import shutil
import os
import random
from PIL import Image

In [31]:
#get frames

import cv2
import numpy as np
from numpy import asarray
 
frames = []
#this video resolution reduced by .25
path = "/content/drive/MyDrive/digitRecon/testVids/readingTest2.MP4"
cap = cv2.VideoCapture(path)
ret = True
while ret:
    ret, img = cap.read() # read one frame from the 'capture' object; img is (H, W, C)
    if ret:
        frames.append(img)

In [32]:
#chop down frames
nth = 40 # n frames to get data from in video
xframes = frames[nth-1::nth]
print(len(xframes))

134


In [33]:
#image display
def displayIm(cv2Image):
  im = Image.fromarray(cv2Image)
  im = im.convert('RGB')
  return im

In [34]:
from PIL import ImageOps
from PIL import ImageEnhance

#image processing
def imageProcVariable(CV2im):
  #convert to pil
  im_pil = Image.fromarray(CV2im)
  im_pil = im_pil.convert('RGB')

  #greyscale
  gray_image = ImageOps.grayscale(im_pil)

  #invert
  inv = ImageOps.invert(gray_image)

  #enhance contrast
  enh = ImageEnhance.Contrast(inv)
  enh = enh.enhance(2)

  #binarize
  threshold = 118
  thresh = enh.point( lambda p: 255 if p > threshold else 0 )
  bw = thresh.convert('1')

  return bw

In [35]:
#text formatting
import re
def formatOCR(info):
  #replace 'B' and 'O' if mistaken
  ex = info.replace('B','8')
  ex = ex.replace('O','0')

  #remove any extra characters
  rex = re.sub(r"[^a-zA-Z0-9.]"," ",ex)

  #remove trailing . or space
  rex = rex.rstrip(" ")
  rex = rex.rstrip(".")

  #replace mid space with '.'
  rex = rex.replace(" ", ".")

  #replace .. with .
  rex = rex.replace("..", ".")
  return rex

def formatSpeed(info):
  rex = re.sub(r"[^0-9]","",info)
  return rex

In [36]:
#lists to store variables in
longs = []
lats = []
sps = []

In [37]:
for img in xframes:
  #get sections of frame that contain the values
  longIm = img[252:264,259:305,:]
  latIm = img[252:264,305:356,:]
  speedIm = img[252:264,380:420,:]

  #image process
  longImProc = imageProcVariable(longIm)
  latImProc = imageProcVariable(latIm)
  speedImProc = imageProcVariable(speedIm)

  #extract text
  extractedLong = pytesseract.image_to_string(longImProc,config='--psm 7 -c tessedit_char_whitelist=NSEW0123456789.')
  extractedLat = pytesseract.image_to_string(latImProc,config='--psm 7 -c tessedit_char_whitelist=NSEW0123456789.')
  extractedSpeed = pytesseract.image_to_string(speedImProc,config='--psm 7 -c tessedit_char_whitelist=0123456789')

  #format text
  longitude = formatOCR(extractedLong)
  latitude = formatOCR(extractedLat)
  speed = formatSpeed(extractedSpeed)

  #append to relevant arrays
  longs.append(longitude)
  lats.append(latitude)
  sps.append(speed)

In [38]:
print("longitudes:", longs)
print("latitudes:", lats)
print("speeds(mph):", sps)

longitudes: ['1.113', 'M1.112', 'Wi.itz', 'M1.182', 'W..4.I.ip', 'Wie.tili', 'Wi.itt', '1.110', '41.110', 'W1.110', '1.109', '1.109', 'W1.109', 'W1.108', '.1.108', 'W1.108', '41.107', '1.107', 'W1.107', 'W1.106', 'Wi.106', '1..106', '12.4.UG', 'W105', 'Wi.105', 'W7.105', 'W1.104', 'W1.104', 'W1.104', 'W1.104', 'W1.103', 'W1.103', 'W1.103', 'W1.103', 'W1.102', 'W1.102', 'W1.102', 'W1.102', 'W1.101', 'W1.101', 'W1.101', 'W1.101', 'W1.101', 'W1.100', '0T.tialal', '11..00', 'Hee', 'TI.0P.e', 'Wl.ore', '41.098', '11.095', '41.G98', '14.09.0', 'etre', 'Wibwiee.ar', 'seers', 'Hi.svre', '91.J0P', '11.076', '11.V6', 'Tow.eo', 'Teo', '', 'Wie', 'W1.695', 'W1.S95', 'W1.095', 'W1.G94', 'W1.094', 'W1.094', 'W1.094', 'W1.094', 'W1.094', 'WT.094', '71.094', 'W1.094', 'W1.094', 'W1.694', 'W1.G93', 'W1.093', '71.093', '71.093', 'W1.093', '41.093', '71.093', 'W1.092', '71.092', 'W1.092', 'W1.G92', 'W1.091', '41.091', 'W1.091', 'W1.091', 'W1.390', '71.090', '71.090', '71.090', 'W1.0s', 'W1.039', 'W1.039'

In [39]:
longVals = ['W', 'E']
latVals = ['N', 'S']

#converts bool list to list of indices that haven't been removed 'False's
def boolToIndex(boolList):
  notRemovedInd = []
  for i in range (0,len(removedBool)):
    if removedBool[i] == False:
      notRemovedInd.append(i)
  return notRemovedInd

#Long-Latitude Reference
Latitude Range: +49.9 -> +60.84
Longitude Range: +1.77 -> -8.62

In [40]:
#filter longitudes and latitudes to those that make sense
#remember if one is incorrect - both will have to go incl speed
#[Note that this notebook does not fix the speed value as not needed for plotting]
#when removing, save index for future removals of other items

removedBool = [False] * len(longs) #array of bools, True if item removed

longVals = ['W', 'E']
latVals = ['N', 'S']
#longitude----------------------------------------
for i in range(0, len(longs)):
  #remove if beginning doesn't begin with W or E
  try:
    if longs[i][0] not in longVals:
      removedBool[i] = True

    #if more than one letter in prediction
    if len(re.sub("[^a-zA-Z]", "", longs[i])) > 1:
      removedBool[i] = True

    #if more than one '.' or none, remove
    if len(re.sub("[^.]", "", longs[i])) > 1:
      removedBool[i] = True
    
    if len(re.sub("[^.]", "", longs[i])) == 0:
      removedBool[i] = True

  except:
    #if its completely empty also remove
    removedBool[i] = True

#remove duplicates
tempList = []
 
for i in range(0, len(longs)):
    if longs[i] not in tempList:
        tempList.append(longs[i])
    else:
      #remove
      removedBool[i] = True

del tempList

#check if drastically different

#convert current bool list to indices of not removed
notRemovedInd = boolToIndex(removedBool)


#check forward values
for i in range(0,len(notRemovedInd)-1):
  #only check ones that haven't been removed
    try:
      if abs(float(longs[notRemovedInd[i]][1:]) - float(longs[notRemovedInd[i+1]][1:])) > 0.003:
        removedBool[notRemovedInd[i]] = True
    except:#item isn't a valid coord
      removedBool[notRemovedInd[i]] = True


#check behind values
for i in range(1,len(notRemovedInd)):
  #only check ones that haven't been removed
    try:
      if abs(float(longs[notRemovedInd[i]][1:]) - float(longs[notRemovedInd[i-1]][1:])) > 0.003:
        removedBool[notRemovedInd[i]] = True
    except:#item isn't a valid coord
      removedBool[notRemovedInd[i]] = True


#checker (print ones not removed yet)
for i in range(0,len(removedBool)):
  if removedBool[i] == False:
    print(longs[i],lats[i])

W1.110 N53.449
W1.109 NS3.448
W1.108 N53.448
W1.107 N53.446
W1.103 N53.442
W1.102 N53.441
W1.101 N53.440
W1.092 N53.4274
W1.083 N53.413
W1.082 N53.411


In [41]:
#change S to 5 in latitude values
#this section (latitudes) may need more formatting for other videos if readings are not as accurate
for i in range(0, len(lats)):
  newLat = []
  for c in range(0,len(lats[i])):
    if c > 6:
      break
    if lats[i][c] == 'S':
      newLat.append('5')
    else:
      newLat.append(lats[i][c])
  lats[i] = ('').join(newLat)

#check forward values
for i in range(0,len(notRemovedInd)-1):
  #only check ones that haven't been removed
    try:
      if abs(float(lats[notRemovedInd[i]][1:]) - float(lats[notRemovedInd[i+1]][1:])) > 0.003:
        removedBool[notRemovedInd[i]] = True
    except:#item isn't a valid coord
      removedBool[notRemovedInd[i]] = True


#check behind values
for i in range(1,len(notRemovedInd)):
  #only check ones that haven't been removed
    try:
      if abs(float(lats[notRemovedInd[i]][1:]) - float(lats[notRemovedInd[i-1]][1:])) > 0.003:
        removedBool[notRemovedInd[i]] = True
    except:#item isn't a valid coord
      removedBool[notRemovedInd[i]] = True


#checker (print ones not removed yet)
for i in range(0,len(removedBool)):
  if removedBool[i] == False:
    print(lats[i])

N53.449
N53.448
N53.448
N53.446
N53.442
N53.441
N53.440
N53.413
N53.411


In [42]:
#convert bool values to relevant +/- values
#Put latitude first
#N = +; S = -; E = +; W = -

filtCords = []
#create list of filtered long/lats
for i in range(0,len(removedBool)):
  if removedBool[i] == False:
    filtLat = lats[i]
    if filtLat[0] == 'N':
      filtLat = float(filtLat[1:])
    else:
      filtLat = - float(filtLat[1:])
    filtLong = longs[i]
    if filtLong[0] == 'E':
      filtLong = float(filtLong[1:])
    else:
      filtLong = - float(filtLong[1:])
    filtCords.append((filtLat,filtLong))

print(filtCords)

[(53.449, -1.11), (53.448, -1.109), (53.448, -1.108), (53.446, -1.107), (53.442, -1.103), (53.441, -1.102), (53.44, -1.101), (53.413, -1.083), (53.411, -1.082)]


In [43]:
#plot route
import folium
import requests

In [44]:
map_object = folium.Map(location=filtCords[0], width = 800, height = 500, zoom_start=14)
folium.TileLayer('stamentoner').add_to(map_object)

In [45]:
#create paths from coords
paths = []
for i in range(0, len(filtCords)-1):
  path = [filtCords[i], filtCords[i+1]]
  paths.append(path)

for p in paths:
  folium.PolyLine(p,
                  color='red',
                  weight=10,
                  opacity=0.8).add_to(map_object)

In [46]:
map_object